In [25]:
# Standard library
import os
import datetime
import smtplib
from email.message import EmailMessage

# Third-party libraries
import pandas as pd
import yfinance as yf
from matplotlib import pyplot as plt
import mplcyberpunk
from dotenv import load_dotenv

In [26]:
# ==========================================
# 0. Load Secured Environment Variables
# ==========================================
load_dotenv()
SENDER_EMAIL = os.environ.get('EMAIL_USER', '').strip()
APP_PASSWORD = os.environ.get('EMAIL_PASS', '').strip()
RECIPIENT_EMAIL = os.environ.get('EMAIL_TO', '').strip()

# ==========================================
# 1. Configuration & Data Fetching
# ==========================================
ticker_map = {
    'EURUSD=X': 'EURUSD',
    'ZW=F': 'Wheat',
    'ZS=F': 'Soybean',
    'ZC=F': 'Corn',
    'NG=F': 'NatGas',
    'CO2.L': 'Carbon_EU',
    'BDRY': 'Freight'
}

assets = list(ticker_map.keys())
today = datetime.datetime.now()
start_date = today - datetime.timedelta(days=365)

data = yf.download(assets, start=start_date, end=today)['Close']
data = data.rename(columns=ticker_map).dropna()

# ==========================================
# 2. Financial Conversion
# ==========================================
commodities_usd = ['Wheat', 'Soybean', 'Corn', 'NatGas']
df_eur = data.copy()

for item in commodities_usd:
    df_eur[f"{item}_EUR"] = data[item] / data['EURUSD']

final_cols = ['Wheat_EUR', 'Soybean_EUR', 'Corn_EUR', 'NatGas_EUR', 'Carbon_EU', 'Freight']
df_final = df_eur[final_cols]

# ==========================================
# 3. Risk Analyst Logic
# ==========================================
def analyze_risks(df):
    alerts = []
    # Carbon Risk (CBAM 2026 Policy)
    ma20_carbon = df['Carbon_EU'].rolling(window=20).mean().iloc[-1]
    current_carbon = df['Carbon_EU'].iloc[-1]
    if current_carbon > ma20_carbon * 1.05:
        alerts.append(f"⚠️ HIGH CARBON RISK: EU Carbon price (€{current_carbon:.2f}) is 5% above average. Impact on CBAM costs.")

    # Logistic Risk (BDI Index)
    weekly_freight_change = df['Freight'].pct_change(periods=5).iloc[-1]
    if weekly_freight_change > 0.10:
        alerts.append(f"🚢 LOGISTICS ALERT: Freight rose {weekly_freight_change:.1%} this week.")

    # Currency Risk
    monthly_eur_change = data['EURUSD'].pct_change(periods=22).iloc[-1]
    if monthly_eur_change < -0.02:
        alerts.append(f"💶 CURRENCY RISK: EUR/USD dropped {abs(monthly_eur_change):.1%}. Import costs are rising.")

    return alerts

risk_alerts = analyze_risks(df_final)

# ==========================================
# 4. Visualization (Base 100)
# ==========================================
def create_risk_chart(df):
    df_norm = (df / df.iloc[0]) * 100
    plt.style.use("cyberpunk")
    plt.figure(figsize=(10, 6))
    for col in df_norm.columns:
        plt.plot(df_norm.index, df_norm[col], label=col)
    
    plt.title("2026 EU Market Risk Index (Base 100)")
    plt.legend(loc='upper left', fontsize='small')
    mplcyberpunk.add_glow_effects()
    
    chart_path = "risk_report.png"
    plt.savefig(chart_path)
    plt.close()
    return chart_path

# ==========================================
# 5. Automated Secure Email Execution (Com Teste Forçado)
# ==========================================

# NOTE: Altere esta flag para False para voltar ao modo de produção
FORCE_TEST_EMAIL = True 

# A logica de envio agora verifica alertas OU a flag de teste
if risk_alerts or FORCE_TEST_EMAIL: 
    if FORCE_TEST_EMAIL and not risk_alerts:
        risk_alerts.append("🧪 MODO DE TESTE ATIVADO: Este é um e-mail de verificação de infraestrutura, nenhum risco real foi detectado pelo sistema hoje.")
    
    chart_file = create_risk_chart(df_final)
    
    msg = EmailMessage()
    # Adiciona [TESTE] no assunto se for um teste forçado
    subject_prefix = "[TESTE] " if FORCE_TEST_EMAIL else ""
    msg['Subject'] = f"{subject_prefix}🚨 MARKET RISK ALERT - {today.strftime('%Y-%m-%d')}"
    msg['From'] = SENDER_EMAIL
    msg['To'] = RECIPIENT_EMAIL
    
    content = "Summary of Market Analysis for the Dutch Headquarters:\n\n"
    content += "\n".join(risk_alerts)
    content += "\n\nAttached: 12-month EUR-normalized performance index."
    msg.set_content(content)

    with open(chart_file, 'rb') as f:
        msg.add_attachment(f.read(), maintype='image', subtype='png', filename=chart_file)

    # Secure SSL Connection
    context = ssl.create_default_context()
    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
            smtp.login(SENDER_EMAIL, APP_PASSWORD)
            smtp.send_message(msg)
        print("Secure risk report sent successfully.")
    except Exception as e:
        print(f"Error sending email: {e}")
else:
    print("Market stable. No alerts generated.")




[*********************100%***********************]  7 of 7 completed


Secure risk report sent successfully.
